<a href="https://colab.research.google.com/github/Surya-Uday-Singh/CNN_From_Scratch-MNIST-/blob/main/MnistCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist

In [2]:
class Conv:
  #This defines a convolutional layer with 3x3 filters

  def __init__(self, num_of_filters):
    self.num_of_filters = num_of_filters

    #This creates a defined 3x3 filter matrice with random numbers
    self.filters = np.random.randn(num_of_filters, 3, 3)/9

  def filter_regions(self, img):

    h, w = img.shape

    for i in range(h - 2):
      for j in range(w - 2):
        img_region = img[i:(i + 3), j:(j + 3)]
        yield img_region, i, j

  def forward_propagate(self, input):

    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_of_filters))

    for img_region, i, j in self.filter_regions(input):
      output[i, j] = np.sum(img_region * self.filters, axis=(1,2))

    return output

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

conv = Conv(8)
print(train_images[0].shape)
output = conv.forward_propagate(train_images[0])
print(output.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
(28, 28)
(26, 26, 8)


In [3]:
class MaxPool:
  def filter_regions(self, img):

    h, w, _ = img.shape
    new_h = h // 2
    new_w = w // 2

    for i in range(new_h):
      for j in range(new_w):
        img_region2x2 = img[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
        yield img_region2x2, i, j


  def forward_propagate(self, input):

    h, w, num_of_filters = input.shape
    output = np.zeros((h//2, w//2, num_of_filters))

    for img_region2x2, i, j in self.filter_regions(input):
      output[i, j] = np.amax(img_region2x2, axis=(0, 1))

    return output


In [6]:
class Softmax:

  def __init__(self, input_len, nodes):
    self.weights = np.random.randn(input_len, nodes)/ input_len
    self.biases = np.zeros(nodes)

  def activation(self, input):

    self.last_input_shape = input.shape
    input = input.flatten()
    self.last_input = input


    input_len, nodes = self.weights.shape

    forward_propagate = np.dot(input, self.weights) + self.biases
    self.last_forward = forward_propagate
    exp = np.exp(forward_propagate)
    return exp/ np.sum(exp, axis = 0)


  def backprop(self, d_l_d_a, alpha):

    for i, grad in enumerate(d_l_d_a):
      if grad == 0:
        continue

      # e^z
      z_exp = np.exp(self.last_forward)

      # Sum of all e^totals
      S = np.sum(z_exp)

      # Gradients of out[i] against totals(totals is WX + b or self.last_forward)
      d_a_d_z = -z_exp[i] * z_exp / (S ** 2)
      # i == correct class, that index has a diff backprop, we change that index value only
      d_a_d_z[i] = z_exp[i] * (S - z_exp[i])/ (S ** 2)

      # Gradients of z against weights/biases/input
      d_z_d_w = self.last_input
      d_z_d_b = 1
      d_z_d_input = self.weights

      #
      d_l_d_z = grad * d_a_d_z

      #gradients of loss wrt weights, biases, inputs
      d_l_d_w =  d_z_d_w[np.newaxis].T @ d_l_d_z[np.newaxis]
      d_l_d_b = d_l_d_z * d_z_d_b
      d_l_d_input = d_z_d_input @ d_l_d_z

      #update weights
      self.weights -= alpha * d_l_d_w
      self.biases  -= alpha * d_l_d_b

      return d_l_d_input.reshape(self.last_input_shape)

In [7]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images_min = train_images[:2000]
train_labels_min = train_labels[:2000]



conv = Conv(9)
pool = MaxPool()
softmax = Softmax(13 * 13 * 9, 10)

def forward_propagation(image, label):

  res = conv.forward_propagate((image/255) - 0.5)
  res = pool.forward_propagate(res)
  res = softmax.activation(res)

  cost_funct = -np.log(res[label])
  if np.argmax(res) == label:
    acc = 1
  else:
    acc = 0

  return res, cost_funct, acc

def train (img, label, alpha = 0.005):

  res, cost_func, acc = forward_propagation(img, label)

  grad = np.zeros(10)
  grad[label] = -1/ res[label]

  grad = softmax.backprop(grad, alpha)

  return cost_func, acc

cost = 0
num_of_correct_labels = 0

for i, (img, label) in enumerate(zip(train_images_min, train_labels_min)):
  loss, accuracy = train(img,label)

  cost+= loss
  num_of_correct_labels+= accuracy

  if i % 100 == 99:
    print(
      'Iteration %d: Average Loss %.6f | Accuracy: %d%%' %
      (i + 1, loss / (i+1), (num_of_correct_labels/(i + 1)) * 100)
    )






Iteration 100: Average Loss 0.018660 | Accuracy: 23%
Iteration 200: Average Loss 0.009227 | Accuracy: 32%
Iteration 300: Average Loss 0.005496 | Accuracy: 40%
Iteration 400: Average Loss 0.003614 | Accuracy: 47%
Iteration 500: Average Loss 0.003633 | Accuracy: 51%
Iteration 600: Average Loss 0.002642 | Accuracy: 53%
Iteration 700: Average Loss 0.003069 | Accuracy: 55%
Iteration 800: Average Loss 0.002003 | Accuracy: 57%
Iteration 900: Average Loss 0.002763 | Accuracy: 59%
Iteration 1000: Average Loss 0.001243 | Accuracy: 61%
Iteration 1100: Average Loss 0.001063 | Accuracy: 62%
Iteration 1200: Average Loss 0.000745 | Accuracy: 63%
Iteration 1300: Average Loss 0.001069 | Accuracy: 64%
Iteration 1400: Average Loss 0.000526 | Accuracy: 65%
Iteration 1500: Average Loss 0.000474 | Accuracy: 65%
Iteration 1600: Average Loss 0.000441 | Accuracy: 67%
Iteration 1700: Average Loss 0.000173 | Accuracy: 68%
Iteration 1800: Average Loss 0.000468 | Accuracy: 69%
Iteration 1900: Average Loss 0.000405